# CreativeUncut Scraper

 Marked for To Optimize

## Mounting Google Drive



In [ ]:
# Package for mounting google drive
from google.colab import drive
drive.mount('/content/drive')

## Including necessary packages & Functions




In [ ]:
# Packages for Fetching and parsing web data
from bs4 import BeautifulSoup as bs
import requests

# Packages for managing files
import os
import shutil
import json

# Packages for time management
from datetime import datetime as dati
from pytz import timezone    
import time

# Packages for multiprocessing
from multiprocessing.dummy import Pool
import multiprocessing

# Packages for image handling
import io
from PIL import Image
from PIL import ImageFile
# True a flag to download truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Package for sorting list
from operator import itemgetter


from tqdm.notebook import tqdm
import string

In [ ]:
def dati_now():
  return dati.now(timezone('Asia/Kolkata')).strftime('%Y-%m-%d_%H-%M-%S')

def atoz_folders(Data_folder_Today):
  # Creating folder if not exists
  alphabetfolderlist = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
  alphabetfolderlist.add('1')
  alphabetfolderlist.add('The')
  for i in alphabetfolderlist:
    if not os.path.exists(Data_folder_Today+i):
      os.makedirs(Data_folder_Today+i)
  return alphabetfolderlist

def get_scraper():
  # Initializing scraper with session enabled and proxy disabled and max retries adapters enabled
  adapter = requests.adapters.HTTPAdapter(max_retries=20)

  Scraper = requests.Session()
  Scraper.trust_env = False

  Scraper.mount('https://', adapter)
  Scraper.mount('http://', adapter)
  return Scraper

def check_VGame(check_specific_VG):
  # Checking whether this set contains specific game
  check_specific_VG = check_specific_VG.lower()
  found = 0
  print('\nChecking whether game name "{}" is exist'.format(check_specific_VG))
  for i in all_VG_urls_and_title:
    if check_specific_VG in i[1].lower() or check_specific_VG in i[2].lower():
      found += 1
      print('Found {} :\n\t {}'.format(found, i))
  if found == 0:
    print('Not Found')

def get_VG_info(Root_folder_Today):
  # Checking the availability of target site
  with Scraper.get("https://www.creativeuncut.com/game-art-galleries.html") as scr_response:
    if scr_response.status_code == 200:
      scr_bs = bs(scr_response.content)
      print('Finished Getting Main Page')

      # Finding all videogames div tags
      all_VGs_divs = scr_bs.find_all('div',attrs={'class':'ag'})

      # Truncating first 2 tag because these are just texts
      all_VGs_divs = all_VGs_divs[2:]

      print('Number Of VideoGames : ', len(all_VGs_divs))

      # Creating record for each videogames that contains following
      # id    - id of that videogames
      # title - title of that videogames
      # url   - url of that videogames
      all_VG_urls_and_title = []
      for i in range(len(all_VGs_divs)):
        url   = domain_name + all_VGs_divs[i].a['href']
        title = all_VGs_divs[i].text
        all_VG_urls_and_title.append(["{:03d}".format(i+1),title,url])

      print('\n\nFirst 10 :\n{}'.format('\n'.join( [str(i) for i in all_VG_urls_and_title[:11]]  )))
      print('\n\nLast  10 :\n{}'.format('\n'.join( [str(i) for i in all_VG_urls_and_title[-10:]] )))

      # Writing VideoGames records to a text file
      with open(Root_folder_Today + 'all_posts.txt', 'w', encoding='utf-8') as f1:
        f1.write('\n'.join(['\n\t'.join(i) for i in all_VG_urls_and_title]))
      
      return all_VG_urls_and_title
    else:
      print('Main Link is not working')


def get_headers():

  # List to store progress informations
  current_r = [       # Current progress status
                  [], # 0 - Error Downlaod Page  --- Not Used
                  [], # 1 - Error Download Image
                  [], # 2 - Empty VGs
                  [], # 3 - All Files List
                  0, # 4 - Total Image Count
                  0, # 5 - Total Page Count
                  0, # 6 - Finished VG Count            
  ]

  # Headers to download images
  # This site lets users to download image only if it receives a get request with the "Referer" links to that image's page
  # Ex : 
  # IMG URL               : https://www.creativeuncut.com/art/gallery-32/1bh-logo.jpg
  # IMG Page URL (Referer): https://www.creativeuncut.com/gallery-32/1bh-logo.html

  # Directly requesting IMG URL leads to the image's page (a html)
  # Luckily with "Referer" targets to image's page, it is possible to get actual image(jpg, png, gif)
  Headers = {
              'User-agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.76 Safari/537.36',
              'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
              'Accept-Encoding' : 'gzip,deflate,sdch',
              'Referer' : 'https://www.creativeuncut.com/gallery-32/1bh-logo.html'
            }
  return current_r, Headers



# Function to Scrape every video game's images
def VG_Scrape(VG):

  # Recording time took for each Video Game
  vg_scrape_time = time.time()

  # Assigning folder path
  if VG[1][0].upper() in alphabetfolderlist:
    VG_Initial = VG[1][0].upper()
    if VG[1][:4].upper() == 'THE ':
      VG_Initial = 'The'
  else:
    VG_Initial = '1'

  # Creating folder name for given Video Game
  valid_VG_Title_for_windows = ''
  windows_illegal_chars = {'?', ':', '/', '//','\\', '*', '"', '<', '>', '|'}
  for i in VG[1]:
    if i in windows_illegal_chars:
      valid_VG_Title_for_windows += '_'
    else:  
      valid_VG_Title_for_windows += i

  VG_folder = f'{Data_folder_Today}{VG_Initial}/{valid_VG_Title_for_windows}/'
  
  # Creating folder if not exists
  if not os.path.exists(VG_folder):
    os.makedirs(VG_folder)

  # Checking if this Video game is already downloaded or not
  if not os.path.exists(VG_folder + 'Details.txt'):

    # Initializing counters
    img_no = 0
    current_page_no = 0
    img_url_list = []

    # Getting Video game page URL
    current_page_url = VG[2]

    # Requesting page
    Scr_response = Scraper.get(VG[2],headers=Headers)
    
    # If status code of response is 200 then it is good to go
    if Scr_response.status_code != 200:

      # Error requesting page, so add it to error list
      current_r[0].append([VG[1],VG[2]])
      # Terminating current Video game scraping
      return False
    else:

      # Parsing the page content
      Scr_bs = bs(Scr_response.content, 'html.parser')
      
      # Getting all page URLs
      pages_list = Scr_bs.find('div',attrs={'class':'r_float'})
      next_pages = pages_list.find_all('a',attrs={'class':'gn'})
      no_of_pages = len(next_pages) + 1

      # Traversing through each page
      for _ in range(no_of_pages):

        # Initializing image counter
        all_img_in_current_page = []
        
        # Getting div tag from class "glry"
        # There are two kinds of classes. some VG have glry, while some have gbox. IDK why?
        current_page_img_table = Scr_bs.find('div',attrs={'class':'glry'})
        if current_page_img_table != None:
          all_img_in_current_page = current_page_img_table.find_all('div',attrs={'class':'th'})

        # Getting div tag from class "gbox"
        current_page_img_table_box = Scr_bs.find('div',attrs={'class':'gbox'})
        if current_page_img_table_box != None:
          all_img_in_current_page = current_page_img_table_box.find_all('div',attrs={'class':'gl'})

        # If both divs are not available then add it to "Empy VG" list
        if current_page_img_table == None and current_page_img_table_box == None:
          current_r[2].append([VG[0],img_no,no_of_pages,VG[2],VG[1]])
          print('Empty Posts Found')
          return False

        # Traversing each image
        for each_img in all_img_in_current_page:          
          
          # Skipping image if it's src links to next-page images like "arrow " image that leads to next page instead of an image
          if each_img.div.a.img['src'] not in  {'imgs/next-page.gif', 'imgs/next-page.jpg', 'imgs/next-page.png'}:

            # Getting current IMG url
            ## Marked for To Optimize
            current_img_url = domain_name + each_img.div.a.img['src'].replace('_s.jpg','.jpg').replace('_s.png','.png').replace('_s.gif','.gif')

            # Increamenting image counter
            img_no += 1

            # Getting current image's referer link
            current_img_page_url = domain_name + each_img.div.a['href']

            # Adding image's url to list
            img_url_list.append(current_img_url)
            current_r[3].append(current_img_url)
            
            # Changing headers of request, so to download image
            Headers['Referer'] = current_img_page_url

            # Requesting image
            res = Scraper.get(current_img_url,headers=Headers)
            
            # Assigning image's name
            VG_img_name = VG_folder + "{:03d}.{}.{}".format(img_no, 'P-{:03d}'.format(current_page_no), current_img_url[current_img_url.rfind('/')+1:])

            # Check if given response is actually an image
            if 'image' not in res.headers.get("content-type", ''):
              
              # Not an image's response, so adding to error list
              current_r[1].append([VG[1],VG[2],current_img_url])
              print(current_img_url,'Image NOt found for ',VG_img_name,' ',VG[1])
            else:

              # Good image, so getting it's binary content
              i = Image.open(io.BytesIO(res.content))
              # Saving image to file
              i.save(VG_img_name)
        
        # Increamenting page counter
        current_page_no += 1

        # Check if last page reached
        if no_of_pages < current_page_no:
          # Getting next page url
          next_page_url = domain_name + next_pages[current_page_no - 1]['href']

          # Assigning current page URL
          current_page_url = next_page_url
          
          # Requesting next page
          Scr_response = Scraper.get(next_page_url,headers=Headers)

          # Check if status code is 200
          if Scr_response.status_code != 200:
            # Error requesting page, so add it to error list
            current_r[0].append([VG[1],current_page_url])
            # Terminating current Video game scraping
            return False
          else:
            # Parsing current response
            Scr_bs = bs(Scr_response.content, 'html.parser')

    # Increamenting page, image counter
    current_r[4] += img_no
    current_r[5] += current_page_no
    current_r[6] += 1

    # Calculating time took for each Video Game
    vg_scrape_time = time.time() - vg_scrape_time

    # Creating a report for Scraping VG
    detail_content_raw = [
                          'Title    : {}'.format(VG[1]),
                          'URL      : {}'.format(VG[2]),
                          'Post ID  : {}'.format(VG[0]),
                          'No IMGs  : {}'.format(img_no),
                          'No Pages : {}'.format(current_page_no),
                          'Time Took: {:.2f} Seconds'.format(vg_scrape_time),
                          '\n',
                          'IMG URLS :\n{}'.format('\n'.join(img_url_list))
                          
                          ]
    # Convert report from list to string
    detail_content = '\n'.join(detail_content_raw)

    # Writing report to file
    with open(VG_folder + 'Details.txt','w') as f1:
      f1.write(detail_content)
    
    # Updating status of Scraping
    status(current_r[6], VG[0], img_no, current_page_no, '{:.2f}'.format(vg_scrape_time), VG[1], VG[2])
  
  # If given Video game is already downloaded
  else:
    # Increament have count and add to list
    have[0] += 1
    have[1].append(VG[2])
    print('Have Count = ',have[0])
    
    # Updating progressbar by 1
    # tqdm_bar.update(1)
 
# Function to update status of scraping
def status(a, b, c, d, e, f, g):
  print('{:03d} - ID:{:3s} - IMGs:{:03d} - Dirs:{:03d} - {:6s}Sec - {} - {}'.format(a, b, c, d, e, f, g))

  # Updating progressbar by 1
  # tqdm_bar.update(1)



def get_report():
  # Report of Scraping
  detail_content_raw = [
                            'Scraping took     : {:.2f} Seconds ({})'.format(Scraping_imgs_time, time.strftime('%H-%M-%S', time.gmtime(Scraping_imgs_time))),
                            'Total IMGs        : {}'.format(current_r[4]),
                            'Total Pages       : {}'.format(current_r[5]),
                            'Total Posts       : {}'.format(current_r[6]),
                            'Empty Posts {:03d}  : {}'.format(len(current_r[2]),current_r[2]),
                            'Error Posts {:03d}  : {}'.format(len(current_r[0]),current_r[0]),
                            'Error IMG   {:03d}  : {}'.format(len(current_r[1]),current_r[1]),
                            'Have Posts  {:03d}  : {}'.format(len(have[1]),have[1]),
                        ]
  return '\n'.join(detail_content_raw)


# Function to return file size in human understandable format
def file_size(size):
  n, p, d = 0, 1024, ['Bytes', 'KB', 'MB', 'GB', 'TB']
  while size > p:
    size /= p
    n += 1
  return '{:.2f} {}'.format(size, d[n])


def organise_folders():

  # Function to return a directory index in dictionary format
  def dirs_to_dict(i='', previous=''):
    
    # Creating current directory path
    fp = previous + i + '/'
    
    # Creating a sorted list of "Directories" in current directories
    sorted_dir = sorted([k for k in next(os.walk(fp))[1]])
    
    # Assigning current directory name and path
    temp = {
              keys_list[0] : i,   # Current Directory Name
              keys_list[1] : fp,  # Current Directory Path
            }
    
    # Getting a list of files with full path in current directory
    temp[keys_list[7]] =  [fp+j for j in sorted([k for k in next(os.walk(fp))[2]]) ]  

    # Counting number of files in current directory
    temp[keys_list[2]] = len(temp[keys_list[7]])
    
    # Calculating total size of files in current directory
    temp[keys_list[3]] = sum([os.path.getsize(j) for j in temp[keys_list[7]]])

    # Creating an empty list to store subdirectory's data
    temp[keys_list[8]] = []
    
    # Copying number files in current directory to initialize the calculation of total number of files inside current directory (including files in subdirectory)
    temp[keys_list[4]] = temp[keys_list[2]]
    
    # Inititalizing total directorys count to zero
    temp[keys_list[5]] = 0
    
    # Copying size of files in current directory to initialize the calculation of total size of files inside current directory (including files in subdirectory)
    temp[keys_list[6]] = temp[keys_list[3]]
    
    # Check if leaf directory is reached or not
    # If FALSE skip this part
    # If TRUE go inside 
    if sorted_dir != []:
      
      # Traversing through each directory
      for j in sorted_dir:
      
        # Calling recursive function to it's subdirectorys and gets it's directory index
        temp_dict = dirs_to_dict(j, fp)
        
        # Getting files, directorys, file's size
        temp_Files_count, temp_Folders_count, temp_Size = temp_dict[keys_list[4]], temp_dict[keys_list[5]], temp_dict[keys_list[6]]
        
        # Appending subdirectory index to current directory's directory section
        temp[keys_list[8]].append(temp_dict)
        
        # Increamenting total files count inside current directory
        temp[keys_list[4]]  += temp_Files_count
        
        # Increamenting total directorys count inside current directory
        temp[keys_list[5]]  += temp_Folders_count + 1
        
        # Increamenting total file's size inside current directory
        temp[keys_list[6]]  += temp_Size
    if '[' not in fp:
      rename_list.append([fp, '{} [ {} - {} ]'.format(fp[:-1], temp[keys_list[4]], file_size(temp[keys_list[6]]))] )

    # Adding a key that stores file size in readable format
    temp[keys_list[9]]  = file_size(temp[keys_list[6]])

    # Returning index dictionary of current directory
    return temp

  # Function to rename folders
  def rename_folders(rename_list):
    # If set 1 Rename folders
    # If set 0 Undo Rename Folders
    option = 1
    j, k = 1-option, 1*option
    for i in rename_list:
      os.rename(i[j], i[k])
    print('Folders renamed successfully')
    

  # Assigning keys for dictionary
  use_short_keys = 0

  # IF TRUE use shorter keys, else use longer meaningful keys
  if use_short_keys == 1:
    keys_list = 'FoN#P#_CFi#_CS #_TFi#_TFo#_TS #__Fi #__Fo #_TSR#4'.split('#')
  else:
    keys_list = 'Folder_Name   #Path          #_Current_Files#_Current_Size #_Total_Files  #_Total_Folders#_Total_Size   #__Files       #__Folders     #_TS_Readable  #14'.split('#')


  # List to rename folders
  # 1st value in each item points to "Original Folder Name"
  # 2nd value in each item points to "Renamed  Folder Name"
  rename_list = []

  # Directory name to index
  dir_name = Root_folder_Today[:-1]

  # Calling indexing function and get the directory index
  dir_data = dirs_to_dict(dir_name, '')

  print('JSON data Created')

  # Exclude from rename list
  excluded_rename_list = [Root_folder_Today, Root_folder_Today[:-1]+'/.ipynb_checkpoints/']
  rename_list = [i for i in rename_list if i[0] not in excluded_rename_list]

  rename_folders(rename_list)

  # Calling indexing function and get the directory index
  dir_data = dirs_to_dict(dir_name, '')

  # Showing information of current directory from dictionary
  for i in dir_data:
    if i not in [keys_list[7], keys_list[8]]:
      temp = '{:' + keys_list[10] + 's} -> {}'
      if i != keys_list[6]:
        print(temp.format(i, dir_data[i]))
      else:
        print(temp.format(i, file_size(dir_data[i])))

  # Assigning JSON filename
  json_file_name = '{}/{} - Index.json'.format(dir_name, dir_name)

  # Writing dictionary in JSON format with "Sorted Keys" and "UNICODE Support" and "Indentation"
  with open(json_file_name, 'w') as f1:
    json.dump(dir_data, f1, sort_keys=True, ensure_ascii=False, indent=4)

  print('Directory Index created successfully')


## Initiating Necessary variables

In [ ]:
# Initiating program execution counter
execution_time_full = time.time()

# Assigning folder names
Today_now = dati_now()
Root_folder_Today = f'Docs{Today_now}/'
Data_folder_Today = Root_folder_Today + 'Data/'

# Domain of target site
domain_name = 'https://www.creativeuncut.com/'

alphabetfolderlist = atoz_folders(Data_folder_Today)

Scraper = get_scraper()

# Creating atoz in ascii value
ascii = [i for i in string.ascii_letters]

# Assign number of threads to run
no_of_threads = multiprocessing.cpu_count() * 2

current_r, Headers = get_headers()

In [ ]:
all_VG_urls_and_title = get_VG_info(Root_folder_Today)

In [ ]:
check_VGame('SINoALICE')

## Scraping

In [ ]:
# Creating number of threads 
pool1 = Pool(no_of_threads)

# Creating progressbar wiht "tqdm(full form 'taqaddum' means 'progress' in Arabic)"
# tqdm_bar = tqdm(total=len(all_VG_urls_and_title))


# Initializing Already downloaded list
have = [
           0, # Already have count
          [], # Already have post
        ]

# Recording Scraping time
Scraping_imgs_time = time.time()

# Running created threads
_ = pool1.map(VG_Scrape,all_VG_urls_and_title)
# Closing threads
pool1.close()
pool1.join()

# Closing progress bar
# tqdm_bar.close()

# Calculating Scraping time
Scraping_imgs_time = time.time() - Scraping_imgs_time

In [ ]:
detail_content = get_report()
print(detail_content)

# Writing Report to file
with open(Data_folder_Today+'Project-Details.txt','w') as f1:
  f1.write(detail_content)

## Renaming Folders

  Ex:

    Original Folder name : God of War

    Renamed  Folder name : God of War [ 35 - 3.67 MB]

In [ ]:
organise_folders()

## Archiving Data

In [ ]:
# Calculating full program execution time
execution_time_full = time.time() - execution_time_full
print('Full program took :\n{:.2f} Seconds\n{}'.format(execution_time_full, time.strftime('%HH-%MM-%SS', time.gmtime(execution_time_full))))

In [ ]:
# Archiving useing 7ZIP with no compression
output_archieve_name = '{} - CreativeUncut ({}) ({}) ({}).7z'.format(dati_now(), current_r[6], current_r[4], time.strftime('%HH-%MM-%SS', time.gmtime(execution_time_full)))
!7z a -m0=Copy "$output_archieve_name" "$Root_folder_Today"

## Copying Data

In [ ]:
!rsync -ah --progress "$output_archieve_name" "/content/drive/My Drive/"

## Deleting Files

In [ ]:
a = input('Gonna delete some (Y/N): ')
if 'y' == a.lower():
  print('Deleting : ', Root_folder_Today)
  !rm -rf "$Root_folder_Today"
  print('Deleted : ', Root_folder_Today)
  
  print('Deleting : ', output_archieve_name)
  !rm -rf "$output_archieve_name"
  print('Deleted : ', output_archieve_name)
else:
  print('It"s ok you decided not TO')

# Other Test Features

##Program to Download single image

In [ ]:
#####################Program to Download Single image from creativeuncut.com#########################

# import requests as rq
# import io
# from PIL import Image

# Headers = {
#             'User-agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.76 Safari/537.36',
#             'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#             'Accept-Encoding' : 'gzip,deflate,sdch',
#             'Referer' : 'https://www.creativeuncut.com/gallery-39/got-heavy-mongol-rank-4.html'
#           }
# llink = 'https://www.creativeuncut.com/gallery-39/art/got-heavy-mongol-rank-4.jpg'

# rres = rq.get(llink, headers=Headers)
# i = Image.open(io.BytesIO(rres.content))
# i.save('1.jpg')


Sort by file sizes

In [ ]:
##########################List all files with respective file size#########################

# all_files_with_size = []
# total_size = 0
# for path, dirs, files in os.walk(start_path):
#   for f in files:
#     fp = os.path.join(path, f)
#     total_size += os.path.getsize(fp)
#     all_files_with_size.append([os.path.getsize(fp),f,fp])
# all_files_with_size = sorted(all_files_with_size, key=itemgetter(0), reverse=True)

# n = 10
# print('First {} :\n{}'.format(n, '\n'.join(['{:10s}{}'.format(file_size(i[0]), i[1]) for i in all_files_with_size[:n+1]])))
# print('Last {} :\n{}'.format(n, '\n'.join(['{:10s}{}'.format(file_size(i[0]), i[1]) for i in all_files_with_size[-n:]])))

In [ ]:
########################Function to get total files and total size of given folder ##########################

# def get_size(start_path = '.'):
#   total_size = 0
#   total_count = 0
#   for path, dir, files in os.walk(start_path):
#     for f in files:
#       total_count += 1
#       total_size += os.path.getsize(os.path.join(path, f))
#   print('Total Files Size  = {}\nTotal Files Count = {}'.format(file_size(total_size), total_count))

# get_size(Data_folder_Today)

Copy Speed comparison

In [ ]:
############################Speed comparison of !cp vs !rsync##################################


# !rm "/content/2020-09-11_15-46-29 - CreativeUncut (949) (52401).7z"

# test2 = time.time()
# !cp "/content/drive/My Drive/My Files/Project_Data/2020-09-11_15-46-29 - CreativeUncut (949) (52401).7z" "/content/2020-09-11_15-46-29 - CreativeUncut (949) (52401).7z"
# test2 = time.time() - test2

# !rm "/content/2020-09-11_15-46-29 - CreativeUncut (949) (52401).7z"

# test1 = time.time()
# !rsync -ah --progress "/content/drive/My Drive/My Files/Project_Data/2020-09-11_15-46-29 - CreativeUncut (949) (52401).7z" "/content/2020-09-11_15-46-29 - CreativeUncut (949) (52401).7z"
# test1 = time.time() - test1

# print('rsync took {:.2f} Sec\ncp    took {:.2f} Sec'.format(test1, test2))


##########################Test Output###################################
# sending incremental file list
# 2020-09-11_15-46-29 - CreativeUncut (949) (52401).7z
#           5.15G 100%   57.60MB/s    0:01:25 (xfr#1, to-chk=0/1)
# rsync took 85.52 Sec
# cp    took 77.65 Sec


###########################Test result:################################
#  !cp is faster than !rsync with time gap of approximately 5 seconds
#  it is not much gap and !rsync shows progress
#
#  Between 5sec faster vs visual progress I will go for Visual progress so i am using rsync from now on



In [ ]:
# from IPython.display import display
# import ipywidgets as widgets
# import time

# out = widgets.HTML()
# display(out)

# for i in range(50):
#   time.sleep(1)
#   out.value='''
#   <html>
#   <style>
#   table.blueTable {
#     border: 1px solid #1C6EA4;
#     background-color: #EEEEEE;
#     width: 100%;
#     text-align: left;
#     border-collapse: collapse;
#   }
#   table.blueTable td, table.blueTable th {
#     border: 1px solid #AAAAAA;
#     padding: 3px 2px;
#   }
#   table.blueTable tbody td {
#     font-size: 13px;
#   }
#   table.blueTable tr:nth-child(even) {
#     background: #D0E4F5;
#   }
#   table.blueTable thead {
#     background: #1C6EA4;
#     background: -moz-linear-gradient(top, #5592bb 0%, #327cad 66%, #1C6EA4 100%);
#     background: -webkit-linear-gradient(top, #5592bb 0%, #327cad 66%, #1C6EA4 100%);
#     background: linear-gradient(to bottom, #5592bb 0%, #327cad 66%, #1C6EA4 100%);
#     border-bottom: 2px solid #444444;
#   }
#   table.blueTable thead th {
#     font-size: 15px;
#     font-weight: bold;
#     color: #FFFFFF;
#     border-left: 2px solid #D0E4F5;
#   }
#   table.blueTable thead th:first-child {
#     border-left: none;
#   }

#   table.blueTable tfoot {
#     font-size: 14px;
#     font-weight: bold;
#     color: #FFFFFF;
#     background: #D0E4F5;
#     background: -moz-linear-gradient(top, #dcebf7 0%, #d4e6f6 66%, #D0E4F5 100%);
#     background: -webkit-linear-gradient(top, #dcebf7 0%, #d4e6f6 66%, #D0E4F5 100%);
#     background: linear-gradient(to bottom, #dcebf7 0%, #d4e6f6 66%, #D0E4F5 100%);
#     border-top: 2px solid #444444;
#   }
#   table.blueTable tfoot td {
#     font-size: 14px;
#   }
#   table.blueTable tfoot .links {
#     text-align: right;
#   }
#   table.blueTable tfoot .links a{
#     display: inline-block;
#     background: #1C6EA4;
#     color: #FFFFFF;
#     padding: 2px 8px;
#     border-radius: 5px;
#   }
#   </style>

#   <body>
#   <table class="blueTable">
#   <thead>
#   <tr>
#   <th>head''' + str(i) + '''</th>
#   <th>head2</th>
#   <th>head3</th>
#   <th>head4</th>
#   <th>head5</th>
#   <th>head6</th>
#   <th>head7</th>
#   </tr>
#   </thead>
#   <tfoot>
#   <tr>
#   <td colspan="7">
#   <div class="links"><a href="#">&laquo;</a> <a class="active" href="#">1</a> <a href="#">2</a> <a href="#">3</a> <a href="#">4</a> <a href="#">&raquo;</a></div>
#   </td>
#   </tr>
#   </tfoot>
#   <tbody>
#   <tr>
#   <td>cell1_1</td>
#   <td>cell2_1</td>
#   <td>cell3_1</td>
#   <td>cell4_1</td>
#   <td>cell5_1</td>
#   <td>cell6_1</td>
#   <td>cell7_1</td>
#   </tr>
#   <tr>
#   <td>cell1_2</td>
#   <td>cell2_2</td>
#   <td>cell3_2</td>
#   <td>cell4_2</td>
#   <td>cell5_2</td>
#   <td>cell6_2</td>
#   <td>cell7_2</td>
#   </tr>
#   <tr>
#   <td>cell1_3</td>
#   <td>cell2_3</td>
#   <td>cell3_3</td>
#   <td>cell4_3</td>
#   <td>cell5_3</td>
#   <td>cell6_3</td>
#   <td>cell7_3</td>
#   </tr>
#   <tr>
#   <td>cell1_4</td>
#   <td>cell2_4</td>
#   <td>cell3_4</td>
#   <td>cell4_4</td>
#   <td>cell5_4</td>
#   <td>cell6_4</td>
#   <td>cell7_4</td>
#   </tr>
#   </tbody>
#   </table>
#   </body>
#   </html>
#   '''
